In [1]:
import pydap.client
import numpy as np
import matplotlib.pyplot as plt

import glob
from subprocess import *
import sys, os
from pylab import *
import pandas as pd 
import datetime
from netCDF4 import Dataset
import xarray as xr


from ctypes import c_float, c_int, cdll, c_short
from numpy.ctypeslib import ndpointer
import time
import multiprocessing 

## Setup C Interface

In [2]:
so_file = '/home/gsilsbe/Documents/cafe/pycafe_multi.so'
lib = cdll.LoadLibrary(so_file)
c_cafe = lib.cafe
c_cafe.restype = ndpointer(dtype=c_float, shape=(8,))

cafe_in = np.empty((9, 4320*2160))
cafe_in[:] = np.NaN
dayofyear = 0

def call_cafe(i):
    return c_cafe(c_float(cafe_in[0, i]), c_float(cafe_in[1, i]), c_float(cafe_in[2, i]),  
                 c_float(cafe_in[3, i]), c_float(cafe_in[4, i]), c_float(cafe_in[5, i]),  
                 c_float(cafe_in[6, i]), c_float(cafe_in[7, i]), c_float(cafe_in[8, i]), c_int(dayofyear))


## Functions called from Wrapper

In [3]:

def nasa_url(d0, d1, ext):
    return u'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/' + d0.strftime('%Y') + '/' + d0.strftime('%j') + '/A' + d0.strftime('%Y') +  d0.strftime('%j') + d0.strftime('%Y') + d1.strftime('%j') + ext

def osu_mld(yyyy, doy0):
    mld_file = '/media/gsilsbe/SILSBE/hycom/osu/monthly/mld.' + yyyy + doy0 + '.hdf'
    mld = xr.open_rasterio(mld_file)
    mld['lon'] = mld['x'] *180/4320 - 180
    mld['lat'] = mld['y'] *90/2160 - 90
    mld = mld.swap_dims({'x': 'lon'}).sel(**{'lon': sorted(mld.lon)}).drop('x')
    mld = mld.swap_dims({'y': 'lat'}).sel(**{'lat': sorted(mld.lat)}).drop('y')
    mld.reindex(lat=list(reversed(mld.lat)))
    mld = mld.squeeze()    
    mld = mld.stack(z=('lat', 'lon'))
    mld = mld.where(mld>0, np.nan)
    return(mld)

def bbw_file(mn):
    return '/media/gsilsbe/SILSBE/common/bbw_400/9km2/bbw_400_' + mn + '_9km2.nc' 


## Giant Wrapper

In [4]:

def cafe(date0, outdir):
    
    date1 = date0 + pd.DateOffset(months=1)  - pd.DateOffset(days=1)
    dayofyear =  np.mean([date0.dayofyear, date1.dayofyear]).astype(int) #average day of year for daylength calcs
    
    outfile = outdir + '/A' + date0.strftime('%Y') + date0.strftime('%j')  + date1.strftime('%Y')  + date1.strftime('%j') + '.L3m_MO_cafe_9km.nc'

    # Stack openDAP into 1D Arrays
    cafe_in[0,:] = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_IOP_adg_443_giop_9km.nc'), engine='netcdf4').adg_443_giop.stack(z=('lat', 'lon'))
    cafe_in[1,:] = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_IOP_aph_443_giop_9km.nc'), engine='netcdf4').aph_443_giop.stack(z=('lat', 'lon'))
    cafe_in[2,:] = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_IOP_bbp_443_giop_9km.nc'), engine='netcdf4').bbp_443_giop.stack(z=('lat', 'lon'))
    cafe_in[3,:] = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_IOP_bbp_s_giop_9km.nc'), engine='netcdf4').bbp_s_giop.stack(z=('lat', 'lon'))
    cafe_in[5,:] = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_CHL_chlor_a_9km.nc'), engine='netcdf4').chlor_a.stack(z=('lat', 'lon'))
    cafe_in[8,:] = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_PAR_par_9km.nc'), engine='netcdf4').par.stack(z=('lat', 'lon'))

    lat = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_CHL_chlor_a_9km.nc'), engine='netcdf4').lat
    lon = xr.open_dataset( nasa_url(date0, date1, '.L3m_MO_CHL_chlor_a_9km.nc'), engine='netcdf4').lon
    attrs_in = xr.open_dataset(nasa_url(date0, date1, '.L3m_MO_PAR_par_9km.nc'), engine='netcdf4').attrs # file attributes

    cafe_in[4,:]  = xr.open_dataset(bbw_file(date0.strftime('%m'))).bbw.stack(z=('lat', 'lon'))   
    cafe_in[6,:] = -np.sort(- np.tile(lat, len(lon)))
    cafe_in[7,:]  = osu_mld(date0.strftime('%Y'), date0.strftime('%j'))

    # Create index of pixels where all data are present
    rowsums = cafe_in.sum(axis=0)
    pixloc = np.argwhere(np.isfinite(rowsums)).flatten()
    
    # Print some information about the run
    print('PAR data file: ', attrs_in['id'])
    print('Valid pixels:', len(pixloc), '% Coverage', round(100 * len(pixloc) / (len(lat) * len(lon)), 1))
    start = time.time()  
    print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    # Run CAFE in Parallel
    a_pool = multiprocessing.Pool(processes=15)
    result = a_pool.map(call_cafe, pixloc)
    a_pool.close()
    a_pool.join()

    # See how many minutes it takes
    print('CAFE C Code Duration (mins):', round((time.time() - start)/60))
    
    # Convert C output to numpy array
    cafe_out = np.array(result)
    cafe_max = np.array([1,10,50,1,10,50,10,10])
    cafe_scale = cafe_max/(2**16-1)
    cafe_offset = 2**15*cafe_scale

    def back_to_numpy(ind):
        var = np.empty(2160 * 4320) * np.nan
        np.put(var, pixloc, cafe_out[:,ind])
        var = np.reshape(var, (2160, 4320))
        var = np.where(var<0, np.nan, var)
        var = np.where(var>cafe_max[ind], np.nan, var)
        return(var)

    npp = back_to_numpy(0)
    gpp = back_to_numpy(1)
    ap = back_to_numpy(2)
    npp_mld = back_to_numpy(3)
    gpp_mld = back_to_numpy(4)
    ap_mld = back_to_numpy(5)
    fnps =  back_to_numpy(6)          
    fnps_mld =  back_to_numpy(7)

    ds = xr.Dataset(
        data_vars=dict(
            npp=(["lat", "lon"], npp),
            gpp=(["lat", "lon"], gpp),
            ap=(["lat", "lon"], ap),
            npp_mld=(["lat", "lon"], npp_mld),
            gpp_mld=([ "lat", "lon"], gpp_mld),
            ap_mld=(["lat", "lon"], ap_mld),
            fnps=(["lat", "lon"], fnps),
            fnps_mld=(["lat", "lon"], fnps_mld),
        ),
        coords={'time':date0, 'lat':lat, 'lon':lon},
        attrs=dict(description = "CAFE model output",
                   time_coverage_start = attrs_in['time_coverage_start'],
                   time_coverage_end = attrs_in['time_coverage_end'],
                   modis_processing_version = attrs_in['processing_version'],
                   par_file = attrs_in['id'],
                   cafe_so_file = 'pycafe_multi.so')
    )
    
    ds.npp.attrs["long_name"] = "net primary production"    
    ds.npp.attrs["units"] = "mol C m-2 d-1"
    ds.gpp.attrs["long_name"] = "gross primary production"    
    ds.gpp.attrs["units"] = "mol C m-2 d-1"
    ds.ap.attrs["long_name"] = "photons absorbed by phytoplankton"    
    ds.ap.attrs["units"] = "mol photons m-2 d-1"
    ds.fnps.attrs["long_name"] = "fraction of non-photosynthetic pigments"    
    ds.fnps.attrs["units"] = "dimensionless"    
    ds.npp_mld.attrs["long_name"] = "net primary production in the mixed layer"    
    ds.npp_mld.attrs["units"] = "mol C m-2 d-1"
    ds.gpp_mld.attrs["long_name"] = "gross primary production in the mixed layer"    
    ds.gpp_mld.attrs["units"] = "mol C m-2 d-1"
    ds.ap_mld.attrs["long_name"] = "photons absorbed by phytoplankton in the mixed layer"    
    ds.ap_mld.attrs["units"] = "mol photons m-2 d-1"
    ds.fnps_mld.attrs["long_name"] = "fraction of non-photosynthetic pigments in the mixed layer"    
    ds.fnps_mld.attrs["units"] = "dimensionless"

    ds.to_netcdf(outfile, encoding={
                 'npp': {"dtype": 'short', "scale_factor": cafe_scale[0],"add_offset": cafe_offset[0],"_FillValue": -32767},
                 'gpp': {"dtype": 'short', "scale_factor": cafe_scale[1],"add_offset": cafe_offset[1],"_FillValue": -32767},
                 'ap': {"dtype": 'short', "scale_factor": cafe_scale[2],"add_offset": cafe_offset[2],"_FillValue": -32767},
                 'npp_mld': {"dtype": 'short', "scale_factor": cafe_scale[3],"add_offset": cafe_offset[3],"_FillValue": -32767},
                 'gpp_mld': {"dtype": 'short', "scale_factor": cafe_scale[4],"add_offset": cafe_offset[4],"_FillValue": -32767},
                 'ap_mld': {"dtype": 'short', "scale_factor": cafe_scale[5],"add_offset": cafe_offset[5],"_FillValue": -32767},
                 'fnps': {"dtype": 'short', "scale_factor": cafe_scale[6],"add_offset": cafe_offset[6],"_FillValue": -32767},
                 'fnps_mld': {"dtype": 'short', "scale_factor": cafe_scale[7],"add_offset": cafe_offset[7],"_FillValue": -32767}})


In [5]:
ts0 = pd.date_range("2014-01-01", "2014-12-31", freq="MS")
outdir = '/media/gsilsbe/SILSBE/cafe/modisa/monthly/'

for j in ts0:
    cafe(j, outdir)

/home/gsilsbe/.conda/envs/mab/lib/python3.7/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


PAR data file:  A20140012014031.L3b_MO_PAR.nc/L3/A20140012014031.L3b_MO_PAR.nc
Valid pixels: 4255940 % Coverage 45.6
2021-05-19 23:45:42
CAFE C Code Duration (mins): 45


/home/gsilsbe/.conda/envs/mab/lib/python3.7/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


PAR data file:  A20140322014059.L3b_MO_PAR.nc/L3/A20140322014059.L3b_MO_PAR.nc
Valid pixels: 4437209 % Coverage 47.6
2021-05-20 00:31:54
CAFE C Code Duration (mins): 47


/home/gsilsbe/.conda/envs/mab/lib/python3.7/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


...

PAR data file:  A20142742014304.L3b_MO_PAR.nc/L3/A20142742014304.L3b_MO_PAR.nc
Valid pixels: 4271259 % Coverage 45.8
2021-05-20 06:20:10
CAFE C Code Duration (mins): 46


/home/gsilsbe/.conda/envs/mab/lib/python3.7/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


PAR data file:  A20143052014334.L3b_MO_PAR.nc/L3/A20143052014334.L3b_MO_PAR.nc
Valid pixels: 4068937 % Coverage 43.6
2021-05-20 07:06:30
CAFE C Code Duration (mins): 43


/home/gsilsbe/.conda/envs/mab/lib/python3.7/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


PAR data file:  A20143352014365.L3b_MO_PAR.nc/L3/A20143352014365.L3b_MO_PAR.nc
Valid pixels: 4051402 % Coverage 43.4
2021-05-20 07:50:42
CAFE C Code Duration (mins): 43
